### Library Importation

In [55]:
# Import the necessary library
import pandas as pd
import re
import joblib
import nltk
nltk.download("punkt")
nltk.download('stopwords')
#nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Data Inspection

In [7]:
# Load datasets

fake = pd.read_csv(r"C:\Users\USER\Desktop\fake_news_detector\Fake.csv")
real = pd.read_csv(r"C:\Users\USER\Desktop\fake_news_detector\True.csv")

In [8]:
# inspect dataset
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
fake.tail()

,title,text,subject,date
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016"


In [10]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [11]:
fake.describe()

,title,text,subject,date
count,23481,23481,23481,23481
unique,17903,17455,6,1681
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017"
freq,6,626,9050,46


In [12]:
fake.shape

(23481, 4)

In [13]:
fake.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

In [14]:
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [15]:
real.tail()

,title,text,subject,date
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017"


In [16]:
real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [17]:
real.describe()

,title,text,subject,date
count,21417,21417,21417,21417
unique,20826,21192,2,716
top,Factbox: Trump fills top jobs for his administ...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"December 20, 2017"
freq,14,8,11272,182


In [18]:
real.shape

(21417, 4)

In [19]:
real.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

### Data Cleaning

In [21]:
# Check for missing value
fake.isna()

,title,text,subject,date
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
23476,False,False,False,False
23477,False,False,False,False
23478,False,False,False,False
23479,False,False,False,False


In [22]:
# Check the total number of missing value if any
fake.isna().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [23]:
real.isna() # Checks missing value

,title,text,subject,date
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
21412,False,False,False,False
21413,False,False,False,False
21414,False,False,False,False
21415,False,False,False,False


In [24]:
real.isna().sum() # Checks number of missing value

title      0
text       0
subject    0
date       0
dtype: int64

In [25]:
# Checks duplicate
int(fake.duplicated().sum())

3

In [26]:
int(real.duplicated().sum()) # Checks duplicate value

206

In [27]:
# Check the duplicate row
fake_dup = fake[fake.duplicated()]
fake_dup

,title,text,subject,date
9942,HILLARY TWEETS MESSAGE In Defense Of DACA…OOPS...,No time to waste we've got to fight with eve...,politics,"Sep 9, 2017"
11446,FORMER DEMOCRAT WARNS Young Americans: “Rioter...,"Who is silencing political speech, physically...",politics,"Mar 10, 2017"
14925,[VIDEO] #BlackLivesMatter Terrorists Storm Dar...,They were probably just looking for a safe sp...,politics,"Nov 16, 2015"


In [28]:
# Actual look into duplicated row
f_i1 = fake.iloc[9941:9943]
f_i2 = fake.iloc[11445:11447]
f_i3 = fake.iloc[14924:14926]

In [29]:
f_i1

,title,text,subject,date
9941,HILLARY TWEETS MESSAGE In Defense Of DACA…OOPS...,No time to waste we've got to fight with eve...,politics,"Sep 9, 2017"
9942,HILLARY TWEETS MESSAGE In Defense Of DACA…OOPS...,No time to waste we've got to fight with eve...,politics,"Sep 9, 2017"


In [30]:
f_i2

,title,text,subject,date
11445,FORMER DEMOCRAT WARNS Young Americans: “Rioter...,"Who is silencing political speech, physically...",politics,"Mar 10, 2017"
11446,FORMER DEMOCRAT WARNS Young Americans: “Rioter...,"Who is silencing political speech, physically...",politics,"Mar 10, 2017"


In [31]:
f_i3

,title,text,subject,date
14924,[VIDEO] #BlackLivesMatter Terrorists Storm Dar...,They were probably just looking for a safe sp...,politics,"Nov 16, 2015"
14925,[VIDEO] #BlackLivesMatter Terrorists Storm Dar...,They were probably just looking for a safe sp...,politics,"Nov 16, 2015"


In [32]:
# Check duplicate row for the real
real_dup = real[real.duplicated()]
real_dup.head()

,title,text,subject,date
445,Senate tax bill stalls on deficit-focused 'tri...,WASHINGTON (Reuters) - The U.S. Senate on Thur...,politicsNews,"November 30, 2017"
778,Trump warns 'rogue regime' North Korea of grav...,BEIJING (Reuters) - U.S. President Donald Trum...,politicsNews,"November 8, 2017"
892,"Republicans unveil tax cut bill, but the hard ...",WASHINGTON (Reuters) - U.S. House of Represent...,politicsNews,"November 2, 2017"
896,Trump taps Fed centrist Powell to lead U.S. ce...,WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"November 2, 2017"
974,"Two ex-Trump aides charged in Russia probe, th...",WASHINGTON (Reuters) - Federal investigators p...,politicsNews,"October 30, 2017"


In [33]:
real_dup.tail()

,title,text,subject,date
21228,France unveils labor reforms in first step to ...,PARIS (Reuters) - French President Emmanuel Ma...,worldnews,"August 31, 2017"
21263,Guatemala top court sides with U.N. graft unit...,GUATEMALA CITY (Reuters) - Guatemala s top cou...,worldnews,"August 29, 2017"
21290,"Europeans, Africans agree renewed push to tack...",PARIS (Reuters) - Europe s big four continen...,worldnews,"August 28, 2017"
21353,Thailand's ousted PM Yingluck has fled abroad:...,BANGKOK (Reuters) - Ousted Thai prime minister...,worldnews,"August 25, 2017"
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017"


In [34]:
# Picks about two rows for confirmation
real.iloc[21407:21409]


,title,text,subject,date
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017"
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017"


In [35]:
real.duplicated().iloc[21407:21409]

21407    False
21408     True
dtype: bool

In [36]:
# Drop na for fake
new_fake = fake.copy()
new_real = real.copy()
new_fake.drop_duplicates(inplace=True)

In [39]:
# Checks duplicated words in fake
int(fake.duplicated().sum())

3

In [40]:
# Confirm duplicates in the new file
int(new_fake.duplicated().sum())

0

### Preprocessing

In [41]:
# Add labels
new_fake["label"] = 0
new_real["label"] = 1

In [42]:
# Combine both fake and real to have a DataFrame
my_df = pd.concat([new_fake, new_real], axis=0)
my_df= my_df.sample(frac=1).reset_index(drop=True)    # Did this to shuffle row

In [43]:
# View data
my_df.head()

,title,text,subject,date,label
0,OOPS! NEW BREAKING POLLS SHOWS DONALD TRUMP IS...,McCain s war hero status is not in question wi...,politics,"Jul 25, 2015",0
1,Parents Ban Black History Month Video From Sc...,Thanks to a bunch of parents who cannot handle...,News,"February 11, 2016",0
2,Bernie Sanders: Trump’s Dreams Of A Second Te...,Sen. Bernie Sanders says that he doesn t think...,News,"April 12, 2017",0
3,PAY 2 PLAY: Democratic Convention Ends Amid Co...,Shawn Helton 21st Century WireThe 2016 Democra...,Middle-east,"July 29, 2016",0
4,Macron hurls challenge to Europe - reform or d...,ATHENS (Reuters) - Europe risks subjugation to...,worldnews,"September 8, 2017",1


In [ ]:
stop_words = set(stopwords.words("english"))

# Creates a function that gets data ready for ml
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z]", " ", text)   # Removes punctuation/numbers
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

In [45]:
# Apply the function to the text column
my_df["text"] = my_df["title"] + " "+ my_df["text"]   # Combines both title and body so algorithm can learn from every bit of text available
my_df["text"] = my_df["text"].apply(clean_text)

In [46]:
my_df["text"]

0        oops new breaking polls shows donald trump pop...
1        parents ban black history month video school w...
2        bernie sanders trump dreams second term go smo...
3        pay play democratic convention ends amid contr...
4        macron hurls challenge europe reform decline a...
                               ...                        
44890    trump says giving peace chance u embassy move ...
44891    conservative columnist tries declare war iran ...
44892    stalled russia sanctions bill hits north korea...
44893    trump says republican make america sick bill w...
44894    obama finally eliminates funding abstinence se...
Name: text, Length: 44895, dtype: object

### Feature Engineering

In [47]:
# Turn text into numeric features that ml algorithm can understand
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(my_df["text"])

y = my_df["label"]

In [48]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5974604 stored elements and shape (44895, 5000)>
  Coords	Values
  (0, 3000)	0.03444843559200797
  (0, 558)	0.0355859484182306
  (0, 3350)	0.03511409913255152
  (0, 4109)	0.03253811995460332
  (0, 1342)	0.06431152819205394
  (0, 4654)	0.27274706926594766
  (0, 3356)	0.03528307352513035
  (0, 2403)	0.11365708471637918
  (0, 2773)	0.41806845422237693
  (0, 4852)	0.15929516663856394
  (0, 2074)	0.23920770835167263
  (0, 4275)	0.036428787875040884
  (0, 3556)	0.029964916578726446
  (0, 197)	0.02172743323841066
  (0, 4827)	0.03972324814942684
  (0, 4048)	0.03295074339960258
  (0, 2951)	0.0274378245183017
  (0, 4020)	0.027804894040975223
  (0, 803)	0.033047831498025176
  (0, 2359)	0.027788522625800845
  (0, 2551)	0.044016281323992726
  (0, 1933)	0.0314721468556533
  (0, 3407)	0.04470493498593655
  (0, 1709)	0.040822195682552305
  (0, 4887)	0.06478890984917812
  :	:
  (44894, 277)	0.047221481139285815
  (44894, 444)	0.0294567401048

### Training and Testing Split

In [49]:
# Splits into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Train Model

In [50]:
# Train model with logistic regression
model = LogisticRegression()
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'




*   Predict with the test data

In [51]:
# Predict with test data to see how model is performing
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 1], shape=(8979,))

*   Compare actual vs predicted

In [52]:
df_results = pd.DataFrame({"Actual (y)": y_test, "Predicted (y)": y_pred})
print("\nActual vs Predicted Values:\n")
df_results.head()


Actual vs Predicted Values:



,Actual (y),Predicted (y)
33834,0,0
2624,0,0
24652,0,0
19545,1,1
37442,0,0


### Model evaluation

In [53]:
# Evaluate model using confusion matrix
y_pred = model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

#confusion_matrix(y_test, y_pred)

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4681
           1       0.99      0.99      0.99      4298

    accuracy                           0.99      8979
   macro avg       0.99      0.99      0.99      8979
weighted avg       0.99      0.99      0.99      8979

Confusion Matrix:
 [[4628   53]
 [  47 4251]]


### Save Model and Vectorizer

In [56]:
# Save model and vectorizer
joblib.dump(model, "fake_news_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']